# Testing on GSM8k Math reasoning

In this notebook, we show how Adala can learn and improve math reasoning skills based on [GSM8k dataset](https://github.com/openai/grade-school-math).

We use a handful of annotated examples from train set, and verify the answer accuracy on the full test set that consists of 1319 examples.

# Baseline

Get the GSM8k datasets with train / test splits:

In [1]:
import pandas as pd
from datasets import load_dataset
gsm8k = load_dataset("gsm8k", "main")

df_train = pd.DataFrame({'question': gsm8k['train']['question'], 'answer': gsm8k['train']['answer']})
df_test = pd.DataFrame({'question': gsm8k['test']['question'], 'answer': gsm8k['test']['answer']})

df_train.head()

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


The following code is used to evaluate the quality of answers by comparing numbers in the outputs:

In [2]:
import re

def extract_and_convert_numbers(text):
    pattern = "\d{1,3}(?:,\d{3})*\.?\d*"
    numbers = re.findall(pattern, text)
    return [int(num.replace(',', '').split('.')[0]) for num in numbers if num.replace(',', '').split('.')[0]]

def evaluate_answers(ground_truth, prediction):
    pred = extract_and_convert_numbers(prediction)
    gt = extract_and_convert_numbers(ground_truth)                             
    return any(p == gt[-1] for p in pred)

Now we can create and execute baseline agent. We start with the naive baseline that just expects answer given the question, by following the prompt template, without any additional instructions:
```
Q: {question}
A: {answer}
```

In [8]:
from rich import print
from adala.agents import Agent
from adala.skills import LinearSkillSet, TransformSkill
from adala.environments import StaticEnvironment
from adala.runtimes import OpenAIChatRuntime


skills = LinearSkillSet(skills=[
    TransformSkill(
        name='math_solver',
        # we start with no instructions then explain how agent can learn more details
        instructions='',
        # instructions=prompt,
        input_template='Q: {question}',
        # here is the baseline established in Kojima et al., 2022 paper
        # output_template='A: Let’s think step by step. {rationale}\nFinal numerical answer:{answer}',
        output_template='A: {answer}',
        instructions_first=False
    )
])

agent = Agent(
    skills=skills,
    
    # this is where agent receives the ground truth signal
    environment = StaticEnvironment(
        df=df_train,
        matching_function=evaluate_answers
    ),
    
    teacher_runtimes={'gpt4': OpenAIChatRuntime(model='gpt-4-1106-preview')},
    default_teacher_runtime='gpt4'
    
)
print(agent)

Agent Instance

Environment: StaticEnvironment
Skills: math_solver
Runtimes: openai
Default Runtime: openai
Default Teacher Runtime: gpt4

In [9]:
# run baseline agent on a test dataset
result_baseline = agent.run(df_test.drop(columns='answer'))

Applying skill: math_solver

100%|█| 1319/1319 [21:58<0


Get baseline evaluation results:

In [10]:
# evaluate baseline results
accuracy = StaticEnvironment(df=df_test, matching_function=evaluate_answers).get_feedback(skills, result_baseline).get_accuracy()

print(f'Baseline accuracy: {accuracy["answer"]}')

Baseline accuracy: 0.4533737680060652

# Improve the baseline

The agent can improve its initial skill by learning new instructions from provided ground truth signals:

In [4]:
agent.learn(batch_size=5, learning_iterations=5)

=> Iteration #0: Getting feedback, analyzing and improving ...

Applying skill: math_solver

100%|█| 5/5 [00:03<00:00, 


Predictions and feedback:

 answer__fb                            question                              answer                               
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Prediction is correct.                John works a job that offers          John makes $10 per hour if he        
                                        performance bonuses.  He makes $80    decides to earn the bonus.           
                                        a day and works for 8 hours.  He                                           
                                        has the option of working hard to                                          
                                        earn the performance bonus of an                                           
                                        extra $20 a day, but the extra                                             
                                        effort results in a 2-hour longer                                          
                                        workday.  How much does John make                                          
                                        per hour if he decides to earn the                                         
                                        bonus?                                                                     
  Prediction is correct.                Nancy agreed to pay 40% of the cost   Nancy will pay $384 each year for    
                                        of her daughter's car insurance,      her daughter's car insurance.        
                                        which costs $80 a month. How much                                          
                                        will Nancy pay each year?                                                  
  Prediction is correct.                There are 200 snakes in a park.       200 - 40 = 160                       
                                        There are three times as many         160/4 = 40                           
                                        pythons as boa constrictors. If       There are 40 rattlesnakes in the     
                                        there 40 boa constrictors and the     park.                                
                                        rest of the snakes are                                                     
                                        rattlesnakes, calculate the total                                          
                                        number of rattlesnakes in the park.                                        
  Prediction is correct.                Abigail spent 60% of her money on                                          
                                        food, and 25% of the remainder on     Let x be the initial amount of       
                                        her phone bill. After spending $20    money Abigail had.                   
                                        on entertainment, she is left with                                         
                                        $40. How much money did Abigail       After spending 60% on food, she is   
                                        have initially?                       left with 40% of x, which is 0.4x.   
                                                                                                                   
                                                                              Out of this 0.4x, she spends 25% on  
                                                                              her phone bill, which is 0.25(0.4x)  
                                                                              = 0.1x.                              
                                                                                                                   
                                                         

...

"{assistant}":
Sure, I’d be happy to help you with this prompt engineering problem. Please provide me with the current prompt and 
the examples with user feedback.

"{user}":

## Current prompt
For each arithmetic word problem provided, you are to:
- Carefully identify and understand the relationships and conditions described within the problem, scrutinizing 
current information as well as any future projections or comparisons.
- Determine the most appropriate mathematical operations needed to find a solution, including but not limited to 
addition, subtraction, multiplication, division, and the application of ratios, percentages, and proportions.
- Execute the calculations with precision, systematically confirming that each step aligns with the problem's 
conditions. Ensure particular accuracy with calculations that involve changes over time, like age or growth rates, 
or percentage-based changes, which require extra attention to accurately apply increases or decreases.
- Verbally detail each step of your calculation process. Clearly articulate the setup of any equations or 
proportions and the rationale for choosing a particular mathematical approach, to create transparency in the logic 
behind each step and elucidate the reasoning that leads to your final answer.
- Critically review each calculation step, ensuring that partial results of multi-step problems are correct before 
proceeding to subsequent calculations. Verify the final answer for consistency with the scenario presented and 
resolve any discrepancies that may arise to conclude with an accurate solution.

## Examples
### Example #0

Q: Brian can only hold his breath underwater for 10 seconds.  He wants to get better, so he starts practicing.  
After a week, he's doubled the amount of time he can do it.  After another week, he's doubled it again from the 
previous week.  The final week, he's increased it by 50% from the previous week.  How long can Brian hold his 
breath for now?

A: 1. Identify and understand the relationships and conditions:
- Brian can hold his breath for 10 seconds.
- After one week of practice, he can hold his breath for 20 seconds.
- After another week of practice, he can hold his breath for 40 seconds.
- After the final week of practice, he can hold his breath for 50% longer than the previous week.

2. Determine the most appropriate mathematical operations:
To find the final amount of time Brian can hold his breath, we will use multiplication and addition.

3. Execute the calculations:
First, we will calculate the amount of time Brian can hold his breath after the final week of practice:
40 seconds x 1.5 = 60 seconds
This means that after the final week of practice, Brian can hold his breath for 60 seconds.

Next, we will add this amount to the previous week's time:
60 seconds + 40 seconds = 100 seconds
This means that after the final week of practice, Brian can hold his breath for 100 seconds.

4. Verbally detail each step:
To find the final amount of time Brian can hold his breath, we first multiplied the previous week's time (40 
seconds) by 1.5 to account for the 50% increase. This gave us 60 seconds. Then, we added this amount to the 
previous week's time to get a final result of 100 seconds.

5. Critically review each calculation step:
The calculations appear to be correct. We multiplied the previous week's time by 1.5 to account for the 50% 
increase, and then added this amount to the previous week's time to get the final result of 100 seconds.

6. Verify the final answer:
The final answer of 100 seconds aligns with the given information and the calculations appear to be correct. 
Therefore, we can conclude that after the final week of practice, Brian can hold his breath for 100 seconds.

User feedback: Prediction is correct.


### Example #1

Q: A shop sells laptops at $600 each and a smartphone at $400.  Celine buys two laptops and four smartphones for 
her children. How much change does she get back if she has $3000?

A: 
Step 1: Identify and understand the relationships and conditions described in the problem.
- Celine is buying two laptops and four smartphones for her children.
- The laptop

"{assistant}":
## Analysis of Incorrect Predictions

### Current Prompt Analysis
The instructions in the current prompt effectively guide the model to follow a systematic approach to solving 
arithmetic word problems, including understanding the conditions, determining the operations, executing 
calculations, detailing each step, reviewing calculations, and verifying the final answer. This thorough approach 
is appropriate for ensuring accuracy and clarity in the model's explanations.

### Breakdown of Incorrect Predictions
In reviewing Example #4, it is clear that the model failed to correctly calculate the total number of pills taken 
in a month. The model's failure appears to be due to a misunderstanding of the calculations needed, specifically 
when determining the number of pills taken each week.

The model erred in its initial calculation by stating "2 weeks x (2 multivitamins + 3 calcium supplements per day) 
= 10 pills per week" when it should have calculated the daily intake and then multiplied by the number of days in 
the two weeks. This indicates an oversight in the comprehension of the problem structure, as well as a failure in 
properly executing multi-step calculations.

### Suggested Changes to the Prompt
To mitigate this kind of error and improve the model's performance, the following changes could be made to the 
prompt:

1. Emphasize the importance of determining the correct units and quantities to be used in calculations. Ensure that
daily, weekly, and monthly quantities are accurately established before computations.
2. Strengthen guidance on multi-step calculations, underscoring the importance of maintaining consistency in units 
throughout these steps (e.g., daily doses multiplied by the number of days, rather than multiplying weekly totals).
3. Incorporate a step for the model to rephrase the problem or put it into a simpler form, which could help clarify
the structure of the problem and prevent misunderstandings.
4. Include an instruction for the model to explicitly confirm and state the time frame (e.g., daily, weekly, 
monthly) of each calculation after setting up the problem but before solving it.
5. Add a checkpoint to reassess the logical flow from one step to the next, to ensure that the sequence of 
operations is correct and that no steps have been skipped or incorrectly summarized.

## Revised Prompt
With these suggestions in mind, here is a revised version of the prompt for arithmetic word problems:

For each arithmetic word problem provided, you are to:
- Methodically identify and understand the relationships and conditions within the problem, reformulating or 
simplifying the scenario if necessary to grasp the key elements and structure.
- Confirm and state the units of measurement (e.g., daily, weekly, monthly quantities) to establish a clear 
foundation for calculations after understanding the problem but before proceeding with mathematical operations.
- Determine the most fitting mathematical operations to resolve the question, being particularly vigilant with the 
calculations when dealing with changes over different time frames or scales (daily, weekly, monthly).
- Execute the calculations with meticulous accuracy, illustrating each step to create transparency in the 
problem-solving process.
- Verbally detail and demonstrate fully each step of your calculation method, providing explicit rationales for 
your choices throughout.
- Conduct a thorough review after each calculation, ensuring that interim results are logical and in the correct 
units before advancing to the next operations.
- Re-examine the entire solution path before finalizing your answer, to ensure that it is sensible and consistent 
with the problem's parameters and that it provides the resolution sought.
- Verify the final answer against the problem to confirm its accuracy, adjusting the solution if discrepancies are 
encountered, and state your final answer clearly, complete with the correct units of measurement.

By i

"{user}":

Now please carefully review your reasoning in Step 1 and help with Step 2: refining the prompt.

## Current prompt
For each arithmetic word problem provided, you are to:
- Carefully identify and understand the relationships and conditions described within the problem, scrutinizing 
current information as well as any future projections or comparisons.
- Determine the most appropriate mathematical operations needed to find a solution, including but not limited to 
addition, subtraction, multiplication, division, and the application of ratios, percentages, and proportions.
- Execute the calculations with precision, systematically confirming that each step aligns with the problem's 
conditions. Ensure particular accuracy with calculations that involve changes over time, like age or growth rates, 
or percentage-based changes, which require extra attention to accurately apply increases or decreases.
- Verbally detail each step of your calculation process. Clearly articulate the setup of any equations or 
proportions and the rationale for choosing a particular mathematical approach, to create transparency in the logic 
behind each step and elucidate the reasoning that leads to your final answer.
- Critically review each calculation step, ensuring that partial results of multi-step problems are correct before 
proceeding to subsequent calculations. Verify the final answer for consistency with the scenario presented and 
resolve any discrepancies that may arise to conclude with an accurate solution.

## Follow this guidance to refine the prompt:

1. The new prompt should should describe the task precisely, and address the points raised in the user feedback.
    
2. The new prompt should be similar to the current instruction, and only differ in the parts that address the 
issues you identified in Step 1.
    Example:
    - Current prompt: "The model should generate a summary of the input text."
    - New prompt: "The model should generate a summary of the input text. Pay attention to the original style."
        
3. Reply only with the new prompt. Do not include input and output templates in the prompt.

Train is done!

The total number of examples needed to teach the agent is _batch_size x learning_iterations_ which is equal to 25 in this case. It is possible to learn more by increasing these parameters.

The new learned instruction is the following:

In [5]:
print(agent.skills['math_solver'].instructions)

For each arithmetic word problem provided, follow these steps:
- Begin by comprehensively identifying and understanding the relationships, conditions, and any given numerical 
information, including the units of measurement (e.g., days, weeks, months) and the related arithmetic operations 
involved in the scenario.
- Clarify the units and timing of events in the word problem, and restate them if necessary to align with the final
units desired for the solution.
- Select the appropriate mathematical operations necessary to obtain the solution. These operations may involve 
addition, subtraction, multiplication, division, or the application of ratios, percentages, and proportions. Pay 
special attention to any transformations or calculations that entail changes over time, such as growth rates, 
aging, or accumulation of resources, ensuring proper application of increases, decreases, or proportional changes.
- Perform the calculations methodically, verifying at each juncture that the operations and results conform to the 
problem's conditions. Clearly articulate each step of your calculation process, demonstrating the setup of 
equations or proportions and explaining the choice of mathematical methods to maintain clarity and logic throughout
your reasoning.
- Examine each intermediate result within multi-step problems, confirming accuracy before proceeding further. 
Reiterate the logical sequence of steps, especially when the problem involves cumulative or sequential changes, to 
ensure continuity and logical progression in the calculation process.
- Conclude by cross-checking the final answer against the conditions and expectations set forth by the problem. 
Resolve any inconsistencies encountered and articulate a coherent, exact solution.

Now let's see how agent performs with the new instructions:

In [7]:
result_new = agent.run(df_test.drop(columns='answer'))
accuracy = StaticEnvironment(df=df_test, matching_function=evaluate_answers).get_feedback(skills, result_new).get_accuracy()
print(f'New accuracy: {accuracy["answer"]}')

Applying skill: math_solver

100%|█| 1319/1319 [1:14:05


New accuracy: 0.7414708112206216